<a href="https://colab.research.google.com/github/Anhbt95/geemap/blob/main/3_using%20earth%20engine%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INFO

Using earth engine data
https://book.geemap.org/chapters/03_gee_data.html

# SETUP

In [ ]:
# install packages
%pip install geemap
%pip install pygis

In [4]:
# load pacakges
import ee
import geemap

In [ ]:
# authorize gee
geemap.ee_initialize()

# earth engine data types

## image

In [ ]:
# earth engine image
## get image
image = ee.Image('USGS/SRTMGL1_003')
image
image.getInfo()

In [9]:
Map = geemap.Map(center=[21.79, 70.87], zoom=3)
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM')
Map

Map(center=[21.79, 70.87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

## collection

In [10]:
# load collection
collection = ee.ImageCollection('COPERNICUS/S2_SR')
collection.limit(5)

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [ ]:
# visualize collection
Map = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR') #copernicus collection
image = collection.median() # calculate median of the collection

# setup visualization
vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(104, 10.2, 10)
Map.addLayer(image, vis, 'Sentinel-2')

#Map

In [24]:
## filter image collection
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01') #image in 2021
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) #could pixel percentage: less than 5%
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(104, 10.2, 10)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[10.2, 104], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

## feature

In [ ]:
# create polygon with spatial info
polygon = ee.Geometry.Polygon(
    [[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]], None, False
)

# add attribute info
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})
polyFeature

# add feature to map
#Map = geemap.Map()
#Map.addLayer(polyFeature, {}, 'feature')
#Map



In [ ]:

# set feature properties
feature = (
    ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))
    .set('genus', 'Sequoia')
    .set('species', 'sempervirens')
)
newDict = {'genus': 'Brachyramphus', 'presence': 1, 'species': 'marmoratus'}
feature = feature.set(newDict)
feature


In [38]:
# get feature properties
prop = feature.get('species')
prop

props = feature.toDictionary()
props

## feature collection

In [ ]:
# load feature collection
Map = geemap.Map()
fc = ee.FeatureCollection('TIGER/2016/Roads')
Map.setCenter(-73.9596, 40.7688, 12)
Map.addLayer(fc, {}, 'Census roads')
#Map

In [14]:
# filter feature collection
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')

# see ee as dataframe
geemap.ee_to_df(states)

feat = states.filter(ee.Filter.eq('NAME', 'Texas'))
Map.addLayer(feat, {}, 'Texas')
Map.centerObject(feat)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
texas = feat.first()
texas.toDictionary()

In [21]:
# filter feature by polygon
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)
Map.addLayer(fc, {}, 'Southeastern U.S.')
Map.addLayer(region, {}, "AOI")
Map.centerObject(fc)
Map


Map(bottom=3666.0, center=[30.543338954230222, -83.51806640625001], controls=(WidgetControl(options=['position…

### visualize feature collections


In [22]:
# us states
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [25]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
#Paints the geometries of a collection onto an image
ee.Image().paint(states, )
image = ee.Image().paint(states, 0, 3)
Map.addLayer(image, {'palette': 'red'}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [26]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000080'}
Map.addLayer(states.style(**style), {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [27]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### style by attribute

In [29]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')
Map

Map(center=[28.00142, -81.7424], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [53]:
# setup type, colors, and fill color
types = [
    "Freshwater Forested/Shrub Wetland",
    "Freshwater Emergent Wetland",
    "Freshwater Pond",
    "Estuarine and Marine Wetland",
    "Riverine",
    "Lake",
    "Estuarine and Marine Deepwater",
    "Other",
]

colors = [
    "#008837",
    "#7FC31C",
    "#688CC0",
    "#66C2A5",
    "#0190BF",
    "#13007C",
    "#007C88",
    "#B28653",
]

#fillColor = [c + "A8" for c in colors]
fillColor = [c for c in colors]


In [58]:
# get feature collection
fc = ee.FeatureCollection("projects/sat-io/open-datasets/NWI/wetlands/FL_Wetlands")

# check feature collection
fc_first = fc.first()
fc_first.toDictionary()

#geemap.ee_to_df(fc) #timeout error

styled_fc = geemap.ee_vector_style(
     fc, column='WETLAND_TY',
     labels=types,
     fillColor=fillColor, color='00000000'
)

In [ ]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')

Map.addLayer(styled_fc, {}, 'NWI')
Map.add_legend(title='Wetland Type', labels=types, colors=colors)
Map

## earth engine catalog